# 07 - Partycjonowanie Danych

Zrozumienie partycjonowania - kluczowy aspekt wydajności w Spark.

**Tematy:**
- Czym są partycje i dlaczego są ważne
- repartition vs coalesce
- Partition pruning - partycjonowanie przy zapisie
- Shuffle - co to jest i jak go minimalizować
- explain() - czytanie planów wykonania
- Broadcast join vs shuffle join
- Bucket join

## 1. Setup

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder \
    .appName("07_Data_Partitioning") \
    .master("spark://spark-master:7077") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.7.1") \
    .config("spark.driver.memory", "6g") \
    .config("spark.executor.memory", "7g") \
    .config("spark.driver.host", "recommender-jupyter") \
    .config("spark.driver.bindAddress", "0.0.0.0") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

jdbc_url = "jdbc:postgresql://postgres:5432/recommender"
properties = {
    "user": "recommender",
    "password": "recommender",
    "driver": "org.postgresql.Driver"
}

ratings = spark.read.jdbc(
    jdbc_url, "movielens.ratings", properties=properties,
    column="user_id", lowerBound=1, upperBound=300000, numPartitions=10
)
movies = spark.read.jdbc(jdbc_url, "movielens.movies", properties=properties)

## 2. Czym są partycje?

Partycja = kawałek danych przetwarzany przez jeden task.

- Więcej partycji → więcej paralelizmu (ale overhead na task)
- Mniej partycji → mniej overhead (ale mniejszy paralelizm)
- Optymalna liczba: 2-4x liczba rdzeni
- Optymalna wielkość: 128 MB - 200 MB per partycja

In [ ]:
# Sprawdź liczbę partycji
print(f"Ratings partitions: {ratings.rdd.getNumPartitions()}")
print(f"Movies partitions: {movies.rdd.getNumPartitions()}")

# Rozmiar partycji (przybliżony)
def partition_sizes(df):
    return df.rdd.mapPartitions(lambda it: [sum(1 for _ in it)]).collect()

sizes = partition_sizes(ratings)
print(f"\nRozmiary partycji ratings: {sizes}")
print(f"Min: {min(sizes)}, Max: {max(sizes)}, Avg: {sum(sizes)/len(sizes):.0f}")

## 3. repartition vs coalesce

- **repartition(n)** - zmienia liczbę partycji (full shuffle), równomierny rozkład
- **coalesce(n)** - zmniejsza liczbę partycji BEZ shuffle (łączy sąsiednie partycje)

**Zasada:** `coalesce` do zmniejszania, `repartition` do zwiększania lub równomiernego rozłożenia.

In [ ]:
# repartition - full shuffle
ratings_20 = ratings.repartition(20)
print(f"Po repartition(20): {ratings_20.rdd.getNumPartitions()} partycji")
print(f"Rozmiary: {partition_sizes(ratings_20)}")

In [ ]:
# coalesce - bez shuffle (tylko zmniejsza)
ratings_4 = ratings.coalesce(4)
print(f"Po coalesce(4): {ratings_4.rdd.getNumPartitions()} partycji")
print(f"Rozmiary: {partition_sizes(ratings_4)}")

# Uwaga: coalesce może dać nierównomierny rozkład!

In [ ]:
# repartition po kolumnie - dane z tym samym kluczem trafiają do tej samej partycji
# Przydatne przed groupBy lub join po tej kolumnie!
ratings_by_user = ratings.repartition(10, "user_id")
print(f"Partycje: {ratings_by_user.rdd.getNumPartitions()}")

# Sprawdź rozkład - dane jednego usera są teraz w jednej partycji
print(f"Rozmiary: {partition_sizes(ratings_by_user)}")

### Zadanie 1
Porównaj czas groupBy("movie_id").count() na:
1. Oryginalnym DataFrame (10 partycji)
2. repartition(10, "movie_id") - repartycjonowanie po kluczu grupowania
3. coalesce(2)

Czy repartition po kluczu pomaga?

In [ ]:
# Twoje rozwiązanie:
import time


## 4. explain() - czytanie planów wykonania

Spark kompiluje operacje DataFrame do planu wykonania. `explain()` pokazuje ten plan.

- **Parsed Logical Plan** - co napisałeś
- **Analyzed Logical Plan** - po rozwiązaniu nazw
- **Optimized Logical Plan** - po optymalizacji Catalyst
- **Physical Plan** - jak Spark to wykona

In [ ]:
# Prosty explain
ratings.filter(col("rating") >= 4.0).explain()

In [ ]:
# Rozszerzony explain - wszystkie poziomy
ratings.filter(col("rating") >= 4.0) \
    .groupBy("movie_id") \
    .count() \
    .explain(mode="extended")

In [ ]:
# Formatted explain - najczytelniejszy
ratings.join(movies, "movie_id") \
    .groupBy("title") \
    .agg(avg("rating").alias("avg_rating")) \
    .orderBy(desc("avg_rating")) \
    .explain(mode="formatted")

### Co szukać w planie:

- **Exchange** = shuffle (kosztowna operacja!)
- **BroadcastHashJoin** = broadcast join (szybki, mały dataset rozesłany do wszystkich workerów)
- **SortMergeJoin** = shuffle join (oba datasety sortowane i łączone)
- **HashAggregate** = agregacja z hashmap
- **Filter** = filtrowanie (dobrze jeśli jest push-down do źródła)
- **Scan** = odczyt danych

In [ ]:
# Porównanie planów: z i bez filtra
print("=== Bez filtra ===")
ratings.groupBy("movie_id").count().explain()

print("\n=== Z filtrem ===")
ratings.filter(col("user_id") < 1000).groupBy("movie_id").count().explain()

## 5. Shuffle - wróg wydajności

Shuffle = przenoszenie danych między partycjami (przez sieć!).

**Operacje powodujące shuffle:**
- `groupBy` / `agg`
- `join` (shuffle join)
- `repartition`
- `distinct`
- `orderBy` (globalne sortowanie)

**Jak minimalizować shuffle:**
1. Filtruj wcześniej (mniej danych do shuffle)
2. Broadcast join zamiast shuffle join
3. repartition po kluczu przed groupBy
4. Używaj coalesce zamiast repartition gdy zmniejszasz

In [ ]:
# spark.sql.shuffle.partitions kontroluje liczbę partycji po shuffle
# Domyślnie 200 - za dużo dla małych danych, za mało dla dużych

print(f"Domyślne shuffle partitions: {spark.conf.get('spark.sql.shuffle.partitions')}")

# Zmniejsz dla naszego datasetu
spark.conf.set("spark.sql.shuffle.partitions", "10")

# Po groupBy - ile partycji?
result = ratings.groupBy("movie_id").count()
print(f"Po groupBy z shuffle.partitions=10: {result.rdd.getNumPartitions()} partycji")

# Przywróć
spark.conf.set("spark.sql.shuffle.partitions", "200")

In [ ]:
# Adaptive Query Execution (AQE) - Spark sam optymalizuje partycje
# Domyślnie włączone w Spark 3.x+
print(f"AQE enabled: {spark.conf.get('spark.sql.adaptive.enabled')}")

# AQE automatycznie:
# - Łączy małe partycje po shuffle (coalesce)
# - Konwertuje sort-merge join na broadcast join gdy dane są małe
# - Optymalizuje skew join (nierównomiernie rozłożone dane)

## 6. Broadcast Join vs Shuffle Join

- **Broadcast join** - mały DataFrame jest kopiowany na każdy executor. Brak shuffle dużego DataFrame.
- **Shuffle join** - oba DataFrames są shufflowane po kluczu joina.

**Zasada:** Jeśli jeden DataFrame jest mały (<10MB domyślnie), Spark automatycznie użyje broadcast.

In [ ]:
# Spark automatycznie broadcastuje movies (mały DataFrame)
# Sprawdź plan - powinien być BroadcastHashJoin
ratings.join(movies, "movie_id").explain()

In [ ]:
# Wymuś broadcast hint
from pyspark.sql.functions import broadcast

# Explicit broadcast
result_broadcast = ratings.join(broadcast(movies), "movie_id")
result_broadcast.explain()

In [ ]:
# Porównanie czasu: broadcast vs shuffle join
import time

# Wyłącz auto broadcast aby wymusić shuffle join
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")

start = time.time()
ratings.join(movies, "movie_id").count()
shuffle_time = time.time() - start
print(f"Shuffle join: {shuffle_time:.2f}s")

# Włącz broadcast
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "10485760")  # 10MB default

start = time.time()
ratings.join(broadcast(movies), "movie_id").count()
broadcast_time = time.time() - start
print(f"Broadcast join: {broadcast_time:.2f}s")
print(f"Broadcast szybszy {shuffle_time/broadcast_time:.1f}x")

### Zadanie 2
Porównaj plany wykonania (explain) dla:
1. `ratings.join(movies, "movie_id")` - automatyczny broadcast
2. Self-join: `ratings.alias("r1").join(ratings.alias("r2"), col("r1.movie_id") == col("r2.movie_id"))` - shuffle join

Zwróć uwagę na Exchange (shuffle) w planie.

In [ ]:
# Twoje rozwiązanie:


## 7. Partycjonowanie przy zapisie

Partycjonowanie danych na dysku pozwala na **partition pruning** - Spark czyta tylko potrzebne partycje.

In [ ]:
# Zapisz dane z partycjonowaniem po roku
ratings_with_year = ratings.withColumn("year", year(col("rating_timestamp")))

# Zapis partycjonowany - tworzy podkatalogi year=2005, year=2006, ...
ratings_with_year.write \
    .partitionBy("year") \
    .mode("overwrite") \
    .parquet("/tmp/ratings_by_year")

In [ ]:
# Odczytaj partycjonowane dane
ratings_partitioned = spark.read.parquet("/tmp/ratings_by_year")

# Partition pruning - Spark czyta TYLKO partycję year=2015
# Sprawdź plan - powinien być PartitionFilters
ratings_partitioned.filter(col("year") == 2015).explain()

In [ ]:
# Porównanie czasu: z i bez partition pruning
import time

# Bez partition pruning (skan całego datasetu)
start = time.time()
ratings.withColumn("year", year(col("rating_timestamp"))) \
    .filter(col("year") == 2015) \
    .count()
no_pruning = time.time() - start

# Z partition pruning (czyta tylko partycję year=2015)
start = time.time()
ratings_partitioned.filter(col("year") == 2015).count()
with_pruning = time.time() - start

print(f"Bez partition pruning: {no_pruning:.2f}s")
print(f"Z partition pruning: {with_pruning:.2f}s")

### Uwaga na liczbę partycji przy zapisie!

Problem: Jeśli mamy 200 shuffle partitions i partycjonujemy po 15 latach → 200 * 15 = 3000 małych plików.

**Rozwiązanie:** coalesce przed zapisem lub użyj `repartition("year")`.

In [ ]:
# Lepszy zapis - kontrolujemy liczbę plików per partycja
ratings_with_year \
    .repartition("year") \
    .write \
    .partitionBy("year") \
    .mode("overwrite") \
    .parquet("/tmp/ratings_by_year_optimized")

## 8. Data Skew - nierównomierny rozkład danych

Jeśli dane są nierównomiernie rozłożone (np. 80% ocen od 10% użytkowników), niektóre partycje będą ogromne a inne prawie puste.

In [ ]:
# Sprawdź skew - rozkład ocen per użytkownik
user_counts = ratings.groupBy("user_id").count()

user_counts.summary().show()

# Top 10 najaktywniejszych
user_counts.orderBy(desc("count")).show(10)

In [ ]:
# Wizualizacja skew - rozkład partycji po repartition(10, "user_id")
ratings_by_user = ratings.repartition(10, "user_id")
sizes = partition_sizes(ratings_by_user)

print("Rozmiary partycji po repartition(user_id):")
for i, s in enumerate(sizes):
    bar = "#" * (s // 50000)
    print(f"  Partition {i:2d}: {s:>8d} rows  {bar}")

print(f"\nSkew ratio: {max(sizes)/min(sizes):.1f}x")

In [ ]:
# Technika: salting - dodaj losowy klucz żeby rozłożyć dane
from pyspark.sql.functions import concat, lit

NUM_SALTS = 5

# Dodaj salt do klucza
ratings_salted = ratings \
    .withColumn("salt", (rand() * NUM_SALTS).cast("int")) \
    .withColumn("user_id_salted", concat(col("user_id"), lit("_"), col("salt")))

# GroupBy z salted key (pierwszy krok)
partial_agg = ratings_salted \
    .groupBy("user_id", "salt") \
    .agg(
        count("*").alias("partial_count"),
        sum("rating").alias("partial_sum")
    )

# Finalna agregacja (drugi krok - już bez skew)
final_agg = partial_agg \
    .groupBy("user_id") \
    .agg(
        sum("partial_count").alias("total_count"),
        round(sum("partial_sum") / sum("partial_count"), 2).alias("avg_rating")
    )

final_agg.orderBy(desc("total_count")).show(10)

## 9. Formaty zapisu i ich wpływ na wydajność

| Format | Kompresja | Column pruning | Predicate pushdown | Opis |
|--------|-----------|----------------|-------------------|------|
| Parquet | Tak | Tak | Tak | Domyślny, najszybszy do analiz |
| ORC | Tak | Tak | Tak | Popularny w Hive |
| CSV | Nie | Nie | Nie | Wolny, duży, czytelny |
| JSON | Nie | Nie | Nie | Wolny, czytelny |

In [ ]:
import time

# Zapisz w różnych formatach
for fmt in ["parquet", "csv", "json"]:
    start = time.time()
    ratings.write.mode("overwrite").format(fmt).save(f"/tmp/ratings_{fmt}")
    write_time = time.time() - start
    print(f"Zapis {fmt}: {write_time:.2f}s")

In [ ]:
# Porównaj odczyt z filtrowaniem
for fmt in ["parquet", "csv", "json"]:
    start = time.time()
    df = spark.read.format(fmt).load(f"/tmp/ratings_{fmt}")
    if fmt == "csv":
        df = df.withColumn("rating", col("rating").cast("double"))
    cnt = df.filter(col("rating") >= 4.5).count()
    read_time = time.time() - start
    print(f"Odczyt + filter {fmt}: {read_time:.2f}s ({cnt} rows)")

## Zadanie końcowe

Zoptymalizuj następujący pipeline:

1. Załaduj ratings i movies
2. Join ratings z movies
3. Filtruj filmy z gatunkiem "Action" wydane po 2010
4. GroupBy per film: avg_rating, count
5. Sortuj po avg_rating DESC

Zoptymalizuj:
- Gdzie umieścić filtr? (przed czy po join?)
- Jaki typ joina wybrać?
- Ile shuffle partitions?
- Porównaj explain() przed i po optymalizacji

In [ ]:
# Wersja nieoptymalna:
result_slow = ratings.join(movies, "movie_id") \
    .filter(col("genres").contains("Action")) \
    .filter(col("title").rlike(r"\(201[0-9]\)")) \
    .groupBy("movie_id", "title") \
    .agg(round(avg("rating"), 2).alias("avg_rating"), count("*").alias("cnt")) \
    .orderBy(desc("avg_rating"))

result_slow.explain()
result_slow.show(10, truncate=False)

In [ ]:
# Twoja zoptymalizowana wersja:


In [ ]:
spark.stop()